# Sort the learning and test flight data_exclude bad data

In [18]:
import os
import pandas as pd
import numpy as np
import math

# matplot
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.floating_axes import GridHelperCurveLinear, FloatingSubplot
import mpl_toolkits.axisartist.grid_finder as gf

# Config
import configparser
import plotly.graph_objects as go

In [19]:
# common_path
common_path = '/path/'


In [20]:
# selected good recording 
recording_df = pd.read_excel('{}/recordings.xlsx'.format(common_path))


In [21]:
recording_df.head()

,Recording,date,time,rec_num,BeeID,condition,comment_Exp,quality,Filename,Calibration,StartFrame,StopFrame,Clutter,Transform,PlotTraj,CommentAnalyse
0,31.0,2021-04-16,16:10:52,1,1R,Learning,NaN,good,20210416_161052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32.0,2021-04-16,17:00:07,1,1R,Test,NaN,good,20210416_170007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2021-04-14,12:03:42,1,22W,Learning,NaN,not_good,20210414_120342,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2021-04-14,12:16:59,1,22W,Test,NaN,good,20210414_121659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,48.0,2021-04-17,17:03:53,2,22W,Learning,NaN,good,20210417_170353,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# All bee list
bee_ids_list = list(recording_df.BeeID)

In [23]:
# removing duplicate bee ids
bee_ids_list = list(dict.fromkeys(bee_ids_list)) # remove the duplicate


In [24]:
print(bee_ids_list)
print(len(bee_ids_list))

['1R', '22W', '28W', '2R', '30W', '31G', '33R', '37G', '44W', '45Y', '72Y', '77Y', '85B']
13


In [25]:
# sorting the bee ids
bee_ids_list.sort()
print(bee_ids_list)
print(len(bee_ids_list))


['1R', '22W', '28W', '2R', '30W', '31G', '33R', '37G', '44W', '45Y', '72Y', '77Y', '85B']
13


In [26]:
# making the empty list with column names 
lf_Reco_sorted = pd.DataFrame(columns=bee_ids_list)
tf_Reco_sorted = pd.DataFrame(columns=bee_ids_list)

# adding sub-headers
tf_Reco_sorted


,1R,22W,28W,2R,30W,31G,33R,37G,44W,45Y,72Y,77Y,85B


In [27]:
lf_Reco_sorted

,1R,22W,28W,2R,30W,31G,33R,37G,44W,45Y,72Y,77Y,85B


In [15]:
recording_df.head()

,Recording,date,time,rec_num,BeeID,condition,comment_Exp,quality,Filename,Calibration,StartFrame,StopFrame,Clutter,Transform,PlotTraj,CommentAnalyse
0,31,2021-04-16,16:10:52,1,1R,Learning,NaN,good,20210416_161052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,2021-04-16,17:00:07,1,1R,Test,NaN,good,20210416_170007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021-04-14,12:16:59,1,22W,Test,NaN,good,20210414_121659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48,2021-04-17,17:03:53,2,22W,Learning,NaN,good,20210417_170353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49,2021-04-17,17:52:44,2,22W,Test,NaN,good,20210417_175244,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Sort the good recordings 

In [32]:
# filling the empty data frame, learning flight and consecutive test flight based on bee id

for ii in range(len(bee_ids_list)): # for all the bee ids
    bee_id = bee_ids_list[ii] # selecting a bee
    bee_data = recording_df.loc[recording_df['BeeID']==bee_id] # getting all the data of the bee
    
    idx = 0 
    for a in range(0,len(bee_data),2): 
                
        if bee_data.quality.iloc[a] == 'good':  # quality manualy added, if it is good
            
            file_LF = bee_data.Filename.iloc[a] # seleting the learning recording 
            lf_Reco_sorted.at[idx,bee_id] = file_LF # adding the recording in the empty data frame of learning flight
            
            if bee_data.quality.iloc[a+1] == 'good':
                file_TF = bee_data.Filename.iloc[a+1] # selecting the test flight recording 
                tf_Reco_sorted.at[idx,bee_id] = file_TF # adding the recording in the empty data frame of test flight
            
            elif bee_data.quality.iloc[a+1] == 'not_good':
                tf_Reco_sorted.at[idx,(bee_id)] = np.nan # add NaN
            
            idx = idx+1
        

        elif bee_data.quality.iloc[a] == 'not_good':  # if data is not good
            lf_Reco_sorted.at[idx,(bee_id)] = np.nan # add NaN
            
            if bee_data.quality.iloc[a+1] == 'good':
                file_TF = bee_data.Filename.iloc[a+1] # selecting the test flight recording 
                tf_Reco_sorted.at[idx,bee_id] = file_TF # adding the recording in the empty data frame of test flight

            elif bee_data.quality.iloc[a+1] == 'not_good':
                tf_Reco_sorted.at[idx,(bee_id)] = np.nan # add NaN
            
            idx = idx+1


There are two conditions for bad quality, 
1. bad recoding 
2. only learning is recorded and not the test flight (inturpted learning flight)

In [33]:
# learning flight recordings - sorted
lf_Reco_sorted 

,1R,22W,28W,2R,30W,31G,33R,37G,44W,45Y,72Y,77Y,85B
0,20210416_161052,NaN,20210414_155821,20210414_163606,NaN,20210414_141848,20210414_123241,NaN,NaN,20210414_150720,NaN,20210417_140607,20210414_171341
1,NaN,20210417_170353,20210416_114344,20210415_171403,NaN,20210416_112759,20210415_160525,20210417_143802,NaN,NaN,20210417_160107,NaN,20210416_171722
2,NaN,NaN,20210417_154738,20210416_152747,NaN,20210417_150451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20210417_161622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Test flight recordings - sorted
tf_Reco_sorted

,1R,22W,28W,2R,30W,31G,33R,37G,44W,45Y,72Y,77Y,85B
0,20210416_170007,20210414_121659,20210414_161434,20210414_165225,NaN,20210414_145838,20210414_134257,20210416_135018,NaN,20210414_154355,20210416_103555,20210417_142728,20210414_173929
1,NaN,20210417_175244,20210416_121026,20210415_172130,NaN,20210416_113809,20210415_171229,20210417_150011,NaN,NaN,20210417_160941,NaN,20210416_173327
2,NaN,NaN,20210417_155715,20210416_160100,NaN,20210417_154338,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20210417_165050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
check_lfReco_sorted = os.path.exists('{}/lf_Reco_sorted.csv'.format(common_path))
if not check_lfReco_sorted:
    lf_Reco_sorted.to_csv('{}/lf_Reco_sorted.csv'.format(common_path),index=False)


check_tfReco_sorted = os.path.exists('{}/tf_Reco_sorted.csv'.format(common_path))
if not check_tfReco_sorted:
    tf_Reco_sorted.to_csv('{}/tf_Reco_sorted.csv'.format(common_path),index=False)
    